In [1]:
import pandas as pd
import numpy as np

In [22]:
df = pd.read_csv("data/train.csv")
pd.options.mode.chained_assignment = None
df_miss = df.copy()

# df.head()

In [23]:
np.random.seed(42)

row_count = df_miss.shape[0]

chosen_idx1 = list(np.random.choice(row_count, replace=False, size=round(row_count*0.12)))
chosen_idx2 = list(np.random.choice(row_count, replace=False, size=round(row_count*0.03)))
chosen_idx3 = list(np.random.choice(row_count, replace=False, size=round(row_count*0.23)))

df_miss["Hepatomegaly"].iloc[chosen_idx1] = None
df_miss["Cholesterol"].iloc[chosen_idx2] = None
df_miss["Copper"].iloc[chosen_idx3] = None

In [ ]:
df_miss.isna().sum()

In [29]:
df_imp1 = df_miss.copy()
df_imp2 = df_miss.copy()
df_imp3 = df_miss.copy()

In [ ]:
df_imp1["Hepatomegaly"] = df_imp1["Hepatomegaly"].fillna(df_imp1["Hepatomegaly"].mode().iat[0])
df_imp1["Cholesterol"] = df_imp1["Cholesterol"].fillna(df_imp1["Cholesterol"].mean())
df_imp1["Copper"] = df_imp1["Copper"].fillna(df_imp1["Copper"].mean())

df_imp1.isna().sum()